<a href="https://colab.research.google.com/github/zhpinkman/hugging-face-projects/blob/master/simple_training_loop_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 6.8 MB 20.7 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 33.5 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 47.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.2 MB/s 
     |████████████████████████████████| 133 kB 50.1 MB/s 
     |████████████████████████████████| 271 kB 50.0 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 


In [ ]:
from transformers import training_args, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

In [ ]:
dataset = load_dataset('glue', 'mrpc')
dataset

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
checkpoint = 'bert-base-cased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
def tokenize(data):
  return tokenizer(data['sentence1'], data['sentence2'], truncation=True)

In [ ]:
tokenized_data = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer) 

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [18]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments('simple training')

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset = tokenized_data['train'], 
    eval_dataset = tokenized_data['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Step,Training Loss
500,0.544200
1000,0.340200


Saving model checkpoint to simple training/checkpoint-500
Configuration saved in simple training/checkpoint-500/config.json
Model weights saved in simple training/checkpoint-500/pytorch_model.bin
tokenizer config file saved in simple training/checkpoint-500/tokenizer_config.json
Special tokens file saved in simple training/checkpoint-500/special_tokens_map.json
Saving model checkpoint to simple training/checkpoint-1000
Configuration saved in simple training/checkpoint-1000/config.json
Model weights saved in simple training/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in simple training/checkpoint-1000/tokenizer_config.json
Special tokens file saved in simple training/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.3715010388830735, metrics={'train_runtime': 394.6669, 'train_samples_per_second': 27.882, 'train_steps_per_second': 3.489, 'total_flos': 419530577771520.0, 'train_loss': 0.3715010388830735, 'epoch': 3.0})

In [ ]:
tokenized_data = tokenized_data.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_data = tokenized_data.rename_column("label", "labels")
tokenized_data.set_format("torch")
tokenized_data["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_data['train'], shuffle=True, batch_size = 8, collate_fn = data_collator)

eval_dataloader = DataLoader(tokenized_data['validation'], batch_size = 8, collate_fn = data_collator)

In [ ]:
for entry in train_dataloader: 
  for key, value in entry.items():
    print(f'{key}: shape {value.shape}')
  break

labels: shape torch.Size([8])
input_ids: shape torch.Size([8, 70])
token_type_ids: shape torch.Size([8, 70])
attention_mask: shape torch.Size([8, 70])


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
outputs = model(**entry)
print(outputs.logits.shape, outputs.loss)

torch.Size([8, 2]) tensor(0.6358, grad_fn=<NllLossBackward0>)


In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
steps = num_epochs * len(train_dataloader)

scheduler = get_scheduler(
    'linear', 
    optimizer = optimizer, 
    num_warmup_steps=0,
    num_training_steps=steps,    
)

print(steps)

1377


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
import torch as th

In [ ]:
device = th.device('cuda') if th.cuda.is_available() else th.device('cpu')
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(steps), leave=False)

model.train()

for epoch in range(num_epochs):
  for entry in train_dataloader:
    optimizer.zero_grad()

    entry = {k: v.to(device) for k, v in entry.items()}
    outputs = model(**entry)

    loss = outputs.loss
    loss.backward()

    optimizer.step()
    scheduler.step()

    
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:
model.eval()

from datasets import load_metric

metric = load_metric('glue', 'mrpc')

for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}

  with th.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  preds = th.argmax(logits, dim = -1)

  metric.add_batch(predictions=preds, references=batch['labels'])

metric.compute()

{'accuracy': 0.8504901960784313, 'f1': 0.8957264957264958}